# Geometric Deep Learning Project - Towards Sparse Hierarchical Graph Classifiers

*Alessia Ruggeri*

### Implementation of the paper *Towards Sparse Hierarchical Graph Classifiers* tested on Enzymes, Proteins and D&D biological datasets using Tensorflow 2.0.

In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
import os,sys,inspect
import networkx as nx
import numpy as np
import scipy
import scipy.sparse as sp
import matplotlib.pyplot as plt
import time

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer, Input, Dense, Flatten, Activation, Dropout, ReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Model
from sklearn.utils import shuffle

from load_data import read_graphfile

np.random.seed(0)

In [0]:
### Unzip datasets folders

# !unzip -o data.zip
!unzip -o data_ENZYMES.zip
# !unzip -o data_PROTEINS.zip
# !unzip -o data_DD.zip
# !unzip -o data_COLLAB.zip

In [0]:
### Load datasets from data.zip

# print("\nLoading ENZYMES...")
# graphs_ENZYMES = read_graphfile(datadir="data", dataname="ENZYMES", max_nodes=None)

# print("\nLoading DD...")
# graphs_DD = read_graphfile(datadir="data", dataname="DD", max_nodes=None)

# print("\nLoading PROTEINS...")
# graphs_PROTEINS = read_graphfile(datadir="data", dataname="PROTEINS", max_nodes=None)

# print("\nLoading COLLAB...")
# graphs_COLLAB = read_graphfile(datadir="data", dataname="COLLAB", max_nodes=None)


### Load datasets from data_DATASET.zip

print("\nLoading ENZYMES...")
graphs_ENZYMES = read_graphfile(datadir="data_ENZYMES", dataname="ENZYMES", max_nodes=None)

# print("\nLoading PROTEINS...")
# graphs_PROTEINS = read_graphfile(datadir="data_PROTEINS", dataname="PROTEINS", max_nodes=None)

# print("\nLoading DD...")
# graphs_DD = read_graphfile(datadir="data_DD", dataname="DD", max_nodes=None)

# print("\nLoading COLLAB...")
# graphs_COLLAB = read_graphfile(datadir="data_COLLAB", dataname="COLLAB", max_nodes=None)

print("\nDone.")

In [0]:
### Functions

def get_numberof_features(dataset_name):
  if dataset_name == "ENZYMES":
    return 18
  elif dataset_name == "PROTEINS":
    return 1
  return None

def preprocess_features(features):
  '''Row-normalize feature matrix and convert it to dense representation'''
  rowsum = np.array(features.sum(1))
  r_inv = np.power(rowsum, -1).flatten()
  r_inv[np.isinf(r_inv)] = 0.
  r_mat_inv = sp.diags(r_inv)
  features = r_mat_inv.dot(features)
  return features

def get_adjacency_matrix(graph):
  '''It returns the adjacency matrix of the graph with inserted self-loops'''
  A = nx.adjacency_matrix(graph)
  A = A + np.eye(A.shape[0])
  A = sp.csr_matrix(A.astype(int))
  return A
  
def get_node_features_matrix(graph):
  '''It returns the node feature matrix of the graph with already preprocessed features'''
  Xdict = nx.get_node_attributes(graph, 'feat')
  X = np.array([Xdict[i] for i in range(nx.number_of_nodes(graph))])
  X = preprocess_features(X)
  X = X.astype(np.float32)
  return X

def get_inverse_degree_matrix(A):
  '''It returns the inverse of the degree matrix constructed from the adjacency matrix of the graph'''
  G = nx.from_numpy_matrix(A.todense())
  degrees = dict(G.degree)
  D = np.zeros(A.shape)
  for i in degrees:
    D[i,i] = degrees[i]
  D = D**(-1)
  D = sp.csr_matrix(D.astype(int))
  return D

def get_graphs_labels(dataset):
  '''It returns the class labels of all the graphs in the dataset'''
  labels = []
  for graph in dataset:
    labels.append(graph.graph['label'])
  labels = np.array([[labels[i]] for i in range(len(labels))])
  return labels

def dot(x, y, sparse=False):
  '''Wrapper for tf.matmul (sparse vs dense)'''
  if sparse:
      res = tf.sparse.sparse_dense_matmul(x, y)
  else:
      res = tf.matmul(x, y)
  return res
  
def convert_sparse_matrix_to_sparse_tensor(coo):
  indices = np.transpose(np.array([coo.row, coo.col]))
  return tf.SparseTensor(indices, coo.data.astype(np.float32), coo.shape)

def convert_nparray_to_sparse_tensor(nparray):
  tf_tensor = tf.constant(nparray)
  idx = tf.where(tf.not_equal(tf_tensor, 0))
  sparse_tensor = tf.SparseTensor(idx, tf.gather_nd(tf_tensor, idx), tf_tensor.get_shape())
  return sparse_tensor
  
def convert_dataset_to_arrays(dataset):
  feat = []
  adj = []
  diag = []
  for graph in dataset:
    X = get_node_features_matrix(graph)
    feat.append(X)
    A = get_adjacency_matrix(graph)
    adj.append(A.tocoo())
    D_inv = get_inverse_degree_matrix(A)
    diag.append(D_inv.tocoo())
#   N = total number of nodes in all the graphs
  X = np.concatenate(feat, axis=0)
#   X_batch = (N, F)
  A = sp.block_diag(adj)
  A = convert_sparse_matrix_to_sparse_tensor(A)
#   A = A.toarray().astype(np.float32)
#   A_batch = (N, N)
  D = sp.block_diag(diag)
  D = convert_sparse_matrix_to_sparse_tensor(D)
#   D = D.toarray().astype(np.float32)
#   D_batch = (N, N)  
  
  return X, A, D
  

In [0]:
### Define the layers

class Convolutional(Layer):
  
  def __init__(self, F, F_1, **kwargs):
    self.F = F
    self.F_1 = F_1
    super(Convolutional, self).__init__(**kwargs)

  def build(self, input_shape):
    self.W1 = self.add_weight(name='W1', 
                             shape=(self.F, F_1),
                             initializer='uniform',
                             regularizer=None,
                             trainable=True)

    self.W2 = self.add_weight(name='W2', 
                             shape=(self.F, F_1),
                             initializer='uniform',
                             regularizer=None,
                             trainable=True)

    super(Convolutional, self).build(input_shape)  # Be sure to call this at the end
    
  def call_one_graph(self, X, A, D_inv):
    
#     A = convert_sparse_matrix_to_sparse_tensor(A)
#     D_inv = convert_sparse_matrix_to_sparse_tensor(D_inv)
#     A = convert_nparray_to_sparse_tensor(A)
#     D_inv = convert_nparray_to_sparse_tensor(D_inv)


    res = dot(A, X, sparse=True)
#     res = (N, F) -> AX
    res = dot(D_inv, res, sparse=True)
#     res = (N, F) -> D**(-1)AX
    res = dot(res, self.W1, sparse=False)
#     res = (N, F_1) -> D**(-1)AXW1

    skip_connection = dot(X, self.W2, sparse=False)
#     skip_connection = (N, F_1) -> XW2
    
    res = tf.math.add(res, skip_connection)
    res = ReLU()(res)
#     res = (N, F_1) -> sigma(D**(-1)AXW1 + XW2)
    
    return res
   
  def call(self, inputs):
    X = inputs[0]
    A = inputs[1]
    D = inputs[2]

    res = self.call_one_graph(X, A, D)
    print(res)

    return res

  def compute_output_shape(self, input_shape):
    return (input_shape[0][0], self.output_dim)


In [296]:
dataset = graphs_ENZYMES
dataset_name = "ENZYMES"
# dataset = graphs_PROTEINS
# dataset_name = "PROTEINS"

labels = get_graphs_labels(dataset)

dataset, labels = shuffle(dataset, labels)

train_n = len(dataset)//100 * 80
val_n = len(dataset)//100 * 15

(x_val, y_val) = dataset[0:val_n], labels[0:val_n]
(x_train, y_train) = dataset[val_n:train_n], labels[val_n:train_n]
(x_test, y_test) = dataset[train_n:], labels[train_n:]

print('train:', len(x_train), len(y_train))
print('val:', len(x_val), len(y_val))
print('test:', len(x_test), len(y_test))

X_train, A_train, D_train = convert_dataset_to_arrays(x_train)
X_val, A_val, D_val = convert_dataset_to_arrays(x_val)
X_test, A_test, D_test = convert_dataset_to_arrays(x_test)

train: 390 390
val: 90 90
test: 120 120


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in reciprocal


In [0]:
# Hyperparameters

epochs=10
batch_size = 32
n_classes=len(np.unique(labels))

dropout = 0.5
learning_rate = 5e-4
reg = l2(5e-4)
momentum = 0.8

F = get_numberof_features(dataset_name)
F_1 = 128

In [298]:
layer = Convolutional(F=F, F_1=F_1, input_shape=(F,))
result = layer([X_val, A_val, D_val])

tf.Tensor(
[[9.5753224e+18 0.0000000e+00 2.4092933e+18 ... 9.4477945e+18
  7.0990480e+18 5.8876781e+18]
 [9.6580980e+18 0.0000000e+00 2.4024359e+18 ... 9.5481777e+18
  7.1600621e+18 5.9334920e+18]
 [9.5739502e+18 0.0000000e+00 2.3790271e+18 ... 9.4629743e+18
  7.0853502e+18 5.8929392e+18]
 ...
 [1.3025328e+19 0.0000000e+00 1.7958751e+18 ... 1.4069941e+19
  9.2075836e+18 9.3753070e+18]
 [1.3097568e+19 0.0000000e+00 1.8128007e+18 ... 1.4148972e+19
  9.2587114e+18 9.4277921e+18]
 [1.3098026e+19 0.0000000e+00 1.8618418e+18 ... 1.4129348e+19
  9.2701716e+18 9.4040713e+18]], shape=(3023, 128), dtype=float32)


In [0]:
# X_in = Input(shape=(F,))
# A_in = Input()

# conv1 = Convolutional(F=F, F_1=F_1)([X_in, A_in])
# conv2 = Convolutional(F=F_1, F_1=F_1)([conv1, A_in])
# conv3 = Convolutional(F=F_1, F_1=F_1)([conv2, A_in])
# flat = Flatten()(conv3)
# dense1 = Dense(256, activation='relu', kernel_regularizer=reg)(flat)
# drop = Dropout(dropout)(dense1)
# dense2 = Dense(n_classes, activation='softmax', kernel_regularizer=reg)(drop)

# model = Model(inputs=[X_in, A_in], outputs=dense2)

model = Sequential()
model.add(Convolutional(F=F, F_1=F_1))
model.add(Convolutional(F=F_1, F_1=F_1))
model.add(Convolutional(F=F_1, F_1=F_1))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer=reg))
model.add(Dropout(dropout))
model.add(Dense(n_classes, activation='softmax', kernel_regularizer=reg))


model.compile(optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [301]:
model.fit([X_train, A_train, D_train], y_train, validation_data=([X_val, A_val, D_val], y_val), epochs=epochs, batch_size=batch_size)

AttributeError: ignored